## Обработка естественного языка

### Предварительная обработка текста

In [ ]:
#исходный текст для анализа
corpus = 'When we were in Paris we visited a lot of museums. We first went to the Louvre, the largest art museum in the world. I have always been interested in art so I spent many hours there. The museum is enourmous, so a week there would not be enough.'

In [ ]:
import nltk
import pandas as pd
import numpy as np

#### Разделение на предложения

In [ ]:
from nltk.tokenize import sent_tokenize

#модель, которая будет делить текст на предложения
nltk.download('punkt')
nltk.download('punkt_tab')

sentences = sent_tokenize(corpus)
sentences

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


['When we were in Paris we visited a lot of museums.',
 'We first went to the Louvre, the largest art museum in the world.',
 'I have always been interested in art so I spent many hours there.',
 'The museum is enourmous, so a week there would not be enough.']

#### Разделение на слова

In [ ]:
from nltk.tokenize import word_tokenize

#первое предложение на слова
print(word_tokenize(sentences[0]))

['When', 'we', 'were', 'in', 'Paris', 'we', 'visited', 'a', 'lot', 'of', 'museums', '.']


In [ ]:
# теперь со всеми предложениями
tokens = []

for sentence in sentences:
    t = word_tokenize(sentence)
    tokens.extend(t)

print(tokens)

['When', 'we', 'were', 'in', 'Paris', 'we', 'visited', 'a', 'lot', 'of', 'museums', '.', 'We', 'first', 'went', 'to', 'the', 'Louvre', ',', 'the', 'largest', 'art', 'museum', 'in', 'the', 'world', '.', 'I', 'have', 'always', 'been', 'interested', 'in', 'art', 'so', 'I', 'spent', 'many', 'hours', 'there', '.', 'The', 'museum', 'is', 'enourmous', ',', 'so', 'a', 'week', 'there', 'would', 'not', 'be', 'enough', '.']


#### Перевод в нижний регистр, удаление стоп-слов и знаков пунктуации

In [ ]:
#модуль стоп-слов
from nltk.corpus import stopwords

#словарь стоп-слов
nltk.download('stopwords')

#set, чтобы оставить только уникальные значения
unique_stops = set(stopwords.words('english'))

#пустой список без стоп-слов
no_stops = []

for token in tokens:

    # переводим все слова в нижний регистр
    token = token.lower()
    if token not in unique_stops and token.isalpha():
        no_stops.append(token)

print(no_stops)

['paris', 'visited', 'lot', 'museums', 'first', 'went', 'louvre', 'largest', 'art', 'museum', 'world', 'always', 'interested', 'art', 'spent', 'many', 'hours', 'museum', 'enourmous', 'week', 'would', 'enough']


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### Лемматизация

In [ ]:
#класс для лемматизации
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
lemmatized = []

# проходимся по всем токенам
for token in no_stops:

    # применяем лемматизацию
    token = lemmatizer.lemmatize(token)

    # добавляем слово после лемматизации в список
    lemmatized.append(token)

print(lemmatized)

['paris', 'visited', 'lot', 'museum', 'first', 'went', 'louvre', 'largest', 'art', 'museum', 'world', 'always', 'interested', 'art', 'spent', 'many', 'hour', 'museum', 'enourmous', 'week', 'would', 'enough']


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


#### Стемминг

In [ ]:
# импортируем класс стеммера Porter и создаём объект этого класса
from nltk.stem import PorterStemmer
porter = PorterStemmer()

# используем list comprehension для стемминга и создание нового списка
stemmed_p = [porter.stem(s) for s in lemmatized]
print(stemmed_p)

['pari', 'visit', 'lot', 'museum', 'first', 'went', 'louvr', 'largest', 'art', 'museum', 'world', 'alway', 'interest', 'art', 'spent', 'mani', 'hour', 'museum', 'enourm', 'week', 'would', 'enough']


In [ ]:
# аналогично импорт Lancaster
from nltk.stem import LancasterStemmer
lancaster = LancasterStemmer()

stemmed_l = [lancaster.stem(s) for s in lemmatized]
print(stemmed_l)

['par', 'visit', 'lot', 'muse', 'first', 'went', 'louvr', 'largest', 'art', 'muse', 'world', 'alway', 'interest', 'art', 'spent', 'many', 'hour', 'muse', 'enourm', 'week', 'would', 'enough']


### Мешок слов (bag of words, bow)

С помощью Counter

In [ ]:
from collections import Counter

# применяем класс Counter к словам после лемматизации
# на выходе возвращается словарь { слово : его частота в тексте }
bow_counter = Counter(lemmatized)
# print(bow_counter)

# функция most_common() упорядочивает словарь по значению
# посмотрим на первые 10 наиболее частотных слов
print(bow_counter.most_common(10))

[('museum', 3), ('art', 2), ('paris', 1), ('visited', 1), ('lot', 1), ('first', 1), ('went', 1), ('louvre', 1), ('largest', 1), ('world', 1)]


С помощью CountVectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# создаём объект этого класса и
# указываем, что хотим перевести слова в нижний регистр, а также
# отфильтровать стоп-слова через stop_words = 'english'
vectorizer = CountVectorizer(analyzer = "word",
                             lowercase = True,
                             tokenizer = None,
                             preprocessor = None,
                             stop_words = 'english',
                             max_features = 5000)

In [ ]:
# применяем этот объект к предложениям (ещё говорят документам)
bow_cv = vectorizer.fit_transform(sentences)

# на выходе получается матрица csr
print(type(bow_cv))

<class 'scipy.sparse._csr.csr_matrix'>


In [ ]:
# преобразуем матрицу csr в привычный формат массива Numpy
# для этого можно использовать .toarray()
print(bow_cv.toarray())

[[0 0 0 0 0 1 0 0 1 1 0 1 0 0 0]
 [1 0 0 0 1 0 1 1 0 0 0 0 0 1 1]
 [1 0 1 1 0 0 0 0 0 0 1 0 0 0 0]
 [0 1 0 0 0 0 0 1 0 0 0 0 1 0 0]]


In [ ]:
# строки это предложения (документы), столбцы - слова (токены)
bow_cv.shape

(4, 15)

In [ ]:
# можно посмотреть на используемые токены (слова)

# здесь числа это не частотность, а просто порядковый номер (индекс)
vocab = vectorizer.vocabulary_
print(vocab)

{'paris': 9, 'visited': 11, 'lot': 5, 'museums': 8, 'went': 13, 'louvre': 6, 'largest': 4, 'art': 0, 'museum': 7, 'world': 14, 'interested': 3, 'spent': 10, 'hours': 2, 'enourmous': 1, 'week': 12}


In [ ]:
# можно вывести слова и без индекса
tokens = vectorizer.get_feature_names_out()
print(tokens)

['art' 'enourmous' 'hours' 'interested' 'largest' 'lot' 'louvre' 'museum'
 'museums' 'paris' 'spent' 'visited' 'week' 'went' 'world']


In [ ]:
# для удобства преобразовать матрицу в датафрейм

# вначале создадим индекс предложений (документов)
index_list = []

# в цикле пройдемся по элементам матрицы, обозначим их через '_'
# функция enumerate задаст каждому элементу индекс, начиная с 0
for i, _ in enumerate(bow_cv):

    # прибавим индекс к слову Sentence
    index_list.append(f'Sentence_{i}')

# print(index_list)

# теперь можно использовать pd.DataFrame()
bow_cv_df = pd.DataFrame(data = bow_cv.toarray(),
                         index = index_list,
                         columns = tokens)
bow_cv_df

,art,enourmous,hours,interested,largest,lot,louvre,museum,museums,paris,spent,visited,week,went,world
Sentence_0,0,0,0,0,0,1,0,0,1,1,0,1,0,0,0
Sentence_1,1,0,0,0,1,0,1,1,0,0,0,0,0,1,1
Sentence_2,1,0,1,1,0,0,0,0,0,0,1,0,0,0,0
Sentence_3,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0


### TF-IDF

#### Способ 1. CountVectorizer + TfidfTransformer

1) Расчет TF, term frequency, частоты слов

In [ ]:
bow_cv

<4x15 sparse matrix of type '<class 'numpy.int64'>'
	with 17 stored elements in Compressed Sparse Row format>

2) Теперь нужно рассчитать IDF

In [ ]:
# импортируем TfidfTransformer (CountVectorizer уже импортирован)
from sklearn.feature_extraction.text import TfidfTransformer

# создадим объект класса TfidfTransformer
tfidf_trans = TfidfTransformer(smooth_idf = True, use_idf = True)

# и рассчитаем IDF слов
tfidf_trans.fit(bow_cv)

# поместим результат в датафрейм
df_idf = pd.DataFrame(tfidf_trans.idf_, index = tokens, columns = ["idf_weights"])
#df_idf

3) Остаётся TF x IDF

In [ ]:
# рассчитаем TF-IDF (по сути умножаем TF на IDF)
tf_idf_vector = tfidf_trans.transform(bow_cv)
tf_idf_vector

<4x15 sparse matrix of type '<class 'numpy.float64'>'
	with 17 stored elements in Compressed Sparse Row format>

In [ ]:
# теперь мы можем посмотреть на показатель TF-IDF для конкретного слова в конкретном документе

# для этого переведем матрицу csr в обычный массив Numpy
df_tfidf = pd.DataFrame(tf_idf_vector.toarray(), columns = vectorizer.get_feature_names_out())

# и траспонируем его (запишем столбцы в виде строк)
print(df_tfidf.T)

              0         1         2         3
art         0.0  0.344315  0.414289  0.000000
enourmous   0.0  0.000000  0.000000  0.617614
hours       0.0  0.000000  0.525473  0.000000
interested  0.0  0.000000  0.525473  0.000000
largest     0.0  0.436719  0.000000  0.000000
lot         0.5  0.000000  0.000000  0.000000
louvre      0.0  0.436719  0.000000  0.000000
museum      0.0  0.344315  0.000000  0.486934
museums     0.5  0.000000  0.000000  0.000000
paris       0.5  0.000000  0.000000  0.000000
spent       0.0  0.000000  0.525473  0.000000
visited     0.5  0.000000  0.000000  0.000000
week        0.0  0.000000  0.000000  0.617614
went        0.0  0.436719  0.000000  0.000000
world       0.0  0.436719  0.000000  0.000000


In [ ]:
# посмотрим сколько слов оставил этот метод после обработки
df_tfidf.T.shape

(15, 4)

#### Способ 2. TfidfVectorizer

In [ ]:
# импортируем класс TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# создаем объект класса TfidfVectorizer
tfIdfVectorizer = TfidfVectorizer(use_idf = True, stop_words= 'english')

# сразу рассчитываем TF-IDF слов
tfIdf = tfIdfVectorizer.fit_transform(sentences)
tfIdf

<4x15 sparse matrix of type '<class 'numpy.float64'>'
	with 17 stored elements in Compressed Sparse Row format>

In [ ]:
# можно посмотреть какие слова остались после фильтрации
print(tfIdfVectorizer.get_feature_names_out())

['art' 'enourmous' 'hours' 'interested' 'largest' 'lot' 'louvre' 'museum'
 'museums' 'paris' 'spent' 'visited' 'week' 'went' 'world']


In [ ]:
# также можно посмотреть IDF слов
tfIdfVectorizer.idf_

array([1.51082562, 1.91629073, 1.91629073, 1.91629073, 1.91629073,
       1.91629073, 1.91629073, 1.51082562, 1.91629073, 1.91629073,
       1.91629073, 1.91629073, 1.91629073, 1.91629073, 1.91629073])

In [ ]:
# через датафрейм мы можем связать слова и их IDF
df_idf = pd.DataFrame(tfIdfVectorizer.idf_, index = tfIdfVectorizer.get_feature_names_out(), columns = ['idf_weights'])
# df_idf.sort_values(by = 'idf_weights', ascending = False)

In [ ]:
# количество предложений (документов) х количество слов
tfIdf.shape

(4, 15)

Расчет значения TF-IDF для каждого слова по каждому тексту

In [ ]:
# и наконец само значение TF-IDF для конкретного слова в конкретном документе
# чем оно уникальнее для конкретного документа, тем выше показатель
df_tfidf = pd.DataFrame(tfIdf.toarray(), columns = tfIdfVectorizer.get_feature_names_out())
print(df_tfidf.T)

              0         1         2         3
art         0.0  0.344315  0.414289  0.000000
enourmous   0.0  0.000000  0.000000  0.617614
hours       0.0  0.000000  0.525473  0.000000
interested  0.0  0.000000  0.525473  0.000000
largest     0.0  0.436719  0.000000  0.000000
lot         0.5  0.000000  0.000000  0.000000
louvre      0.0  0.436719  0.000000  0.000000
museum      0.0  0.344315  0.000000  0.486934
museums     0.5  0.000000  0.000000  0.000000
paris       0.5  0.000000  0.000000  0.000000
spent       0.0  0.000000  0.525473  0.000000
visited     0.5  0.000000  0.000000  0.000000
week        0.0  0.000000  0.000000  0.617614
went        0.0  0.436719  0.000000  0.000000
world       0.0  0.436719  0.000000  0.000000


Расчет среднего значения TF-IDF для каждого слова по всем текстам

In [ ]:
# рассчитаем среднее арифметическое по строкам (axis = 0)
tfIdf.mean(axis = 0)

matrix([[0.18965082, 0.15440359, 0.13136819, 0.13136819, 0.10917983,
         0.125     , 0.10917983, 0.2078122 , 0.125     , 0.125     ,
         0.13136819, 0.125     , 0.15440359, 0.10917983, 0.10917983]])

In [ ]:
# преобразуем матрицу в массив Numpy
np.asarray(tfIdf.mean(axis = 0))

array([[0.18965082, 0.15440359, 0.13136819, 0.13136819, 0.10917983,
        0.125     , 0.10917983, 0.2078122 , 0.125     , 0.125     ,
        0.13136819, 0.125     , 0.15440359, 0.10917983, 0.10917983]])

In [ ]:
# посмотрим сколько измерений
np.asarray(tfIdf.mean(axis = 0)).shape

(1, 15)

In [ ]:
# уберем второе измерение
np.asarray(tfIdf.mean(axis = 0)).ravel()

array([0.18965082, 0.15440359, 0.13136819, 0.13136819, 0.10917983,
       0.125     , 0.10917983, 0.2078122 , 0.125     , 0.125     ,
       0.13136819, 0.125     , 0.15440359, 0.10917983, 0.10917983])

In [ ]:
# снова смотрим на размерность
np.asarray(tfIdf.mean(axis = 0)).ravel().shape

(15,)

In [ ]:
# преобразуем в список
mean_weights = np.asarray(tfIdf.mean(axis = 0)).ravel().tolist()
mean_weights

[0.18965081782108964,
 0.15440359274390048,
 0.13136818731601646,
 0.13136818731601646,
 0.10917982746877804,
 0.125,
 0.10917982746877804,
 0.2078121960479979,
 0.125,
 0.125,
 0.13136818731601646,
 0.125,
 0.15440359274390048,
 0.10917982746877804,
 0.10917982746877804]

In [ ]:
# создаём датафрейм из словаря
mean_weights_df = pd.DataFrame({'term': tfIdfVectorizer.get_feature_names_out(), 'mean_weights': mean_weights})

# сортируем по убыванию 10 слов с максимальным средним TF-IDF
mean_weights_df.sort_values(by = 'mean_weights', ascending = False).reset_index(drop = True).head(10)

,term,mean_weights
0,museum,0.207812
1,art,0.189651
2,enourmous,0.154404
3,week,0.154404
4,hours,0.131368
5,interested,0.131368
6,spent,0.131368
7,lot,0.125000
8,museums,0.125000
9,paris,0.125000


### Дополнительные примеры

#### Косинусное расстояние между текстовыми векторами

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# from scipy.sparse.csr import csr_matrix
# import numpy as np
# import pandas as pd

In [ ]:
# возьмем для простоты два текста (предложения)
text1 = 'all the world’s a stage, and all the men and women merely players'
text2 = 'you must be the change you wish to see in the world'

# объединим их в корпус
corpus = [text1, text2]

# создадим объект класса TfidfVectorizer
tfIdfVectorizer = TfidfVectorizer(use_idf = True, stop_words = 'english')

# на выходе получаем два вектора, где каждое значение - это вес (показатель tf-idf) слова
X = tfIdfVectorizer.fit_transform(corpus)

# преобразуем данные в массив Numpy
print(X.toarray())

[[0.         0.4261596  0.4261596  0.4261596  0.4261596  0.
  0.4261596  0.30321606]
 [0.6316672  0.         0.         0.         0.         0.6316672
  0.         0.44943642]]


In [ ]:
# для удобства можем посмотреть на веса в формате датафрейма
vectors_df = pd.DataFrame(data = X.toarray(),
                          index = ['vector1', 'vector2'],
                          columns = tfIdfVectorizer.get_feature_names_out())
vectors_df

,change,men,merely,players,stage,wish,women,world
vector1,0.000000,0.42616,0.42616,0.42616,0.42616,0.000000,0.42616,0.303216
vector2,0.631667,0.00000,0.00000,0.00000,0.00000,0.631667,0.00000,0.449436


Напомню формулу косинусного расстояния:

$$ \cos \theta ={\mathbf {a} \cdot \mathbf {b} \over \|\mathbf {a} \|\|\mathbf {b} \|} $$

In [ ]:
# возьмем вектора по отдельности
vector1 = X.toarray()[0]
vector2 = X.toarray()[1]

In [ ]:
# вначале выполним операции в числителе формулы
numerator = np.dot(vector1, vector2)

In [ ]:
# теперь займемся знаменателем и
# (1) рассчитаем длины (по большому счету, это теорема Пифагора)
vector1Len = np.linalg.norm(vector1)
vector2Len = np.linalg.norm(vector2)

# (2) перемножим их
denominator = vector1Len * vector2Len

In [ ]:
# посмотрим, чему равен косинус угла между векторами
cosine = numerator/denominator
cosine

0.13627634143908643

In [ ]:
# найдем угол в градусах по его косинусу
# для этого вначале вычислим угол в радианах
angle_radians = np.arccos(cosine)

# затем в градусах
angle_degrees = angle_radians * 360/2/np.pi
round(angle_degrees, 2)

82.17

#### Кластерный анализ текста

In [ ]:
# в тексте ниже две темы: наука о данных и Большой театр (источник: Википедия)
text = '''
Data science is an interdisciplinary field that uses scientific methods, processes, algorithms and systems to extract knowledge and insights from noisy, structured and unstructured data.
It applies knowledge and actionable insights from data across a broad range of application domains.
Data science is related to data mining, machine learning and big data.
The Bolshoi Theatre is a historic theatre in Moscow, Russia.
It was originally designed by architect Joseph Bové, which holds ballet and opera performances.
Before the October Revolution it was a part of the Imperial Theatres of the Russian Empire along with Maly Theatre in Moscow and a few theatres in Saint Petersburg.
Data science is a concept to unify statistics, data analysis, informatics, and their related methods in order to understand and analyze actual phenomena with data.
However, data science is different from computer science and information science.
The main building of the theatre, rebuilt and renovated several times during its history, is a landmark of Moscow and Russia.
On 28 October 2011, the Bolshoi re-opened after an extensive six-year renovation.
'''

In [ ]:
# создадим список из предложений
corpus = []

# для этого в цикле for пройдемся по тексту, разделив его по символу новой строки \n
for line in text.split('\n'):

  # если строка не пустая (т.е. True)
  if line:

    # переводим ее в нижний регистр
    line = line.lower()
    # и добавляем в список
    corpus.append(line)

In [ ]:
corpus

['data science is an interdisciplinary field that uses scientific methods, processes, algorithms and systems to extract knowledge and insights from noisy, structured and unstructured data.',
 'it applies knowledge and actionable insights from data across a broad range of application domains.',
 'data science is related to data mining, machine learning and big data.',
 'the bolshoi theatre is a historic theatre in moscow, russia.',
 'it was originally designed by architect joseph bové, which holds ballet and opera performances.',
 'before the october revolution it was a part of the imperial theatres of the russian empire along with maly theatre in moscow and a few theatres in saint petersburg.',
 'data science is a concept to unify statistics, data analysis, informatics, and their related methods in order to understand and analyze actual phenomena with data.',
 'however, data science is different from computer science and information science.',
 'the main building of the theatre, rebuil

In [ ]:
# применим TfidfVectorizer
tfIdfVectorizer = TfidfVectorizer(use_idf = True, stop_words= 'english')

# на выходе получаем векторы предложений
X = tfIdfVectorizer.fit_transform(corpus)
# print(X.toarray())

In [ ]:
# импортируем алгоритм k-средних из библиотеки sklearn
from sklearn.cluster import KMeans

# так как мы знаем, что темы две, используем гиперпараметр k = 2
kmeans = KMeans(n_clusters = 2, n_init = 10, random_state = 42).fit(X)

In [ ]:
# возьмем новые предложения, одно из области Data Science и два про Большой театр
prediction = ['Many statisticians, including Nate Silver, have argued that data science is not a new field, but rather another name for statistics.',
              'Urusov set up the theatre in collaboration with English tightrope walker Michael Maddox.',
              'Until the mid-1990s, most foreign operas were sung in Russian, but Italian and other languages have been heard more frequently on the Bolshoi stage in recent years.']

# применим две модели, сначала создадим векторы новых предложений (tfIdfVectorizer.transform),
# затем отнесем их к одному из кластеров (kmeans.predict)
kmeans.predict(tfIdfVectorizer.transform(prediction))

array([1, 0, 0], dtype=int32)